# decorator
- 코드 로깅
- 파라미터 유효성 검사
- retry 로직 구현
- 반복작업을 데코레이터에 넣어서 코드 단순화

In [1]:
import time

In [12]:
def perf_time(func):
    def perf_clock(*args):
        start_time = time.perf_counter()
        result = func(*args)
        end_time = time.perf_counter()
        name = func.__name__
        args_str = ', '.join(repr(arg) for arg in args)
        print(f'time: {end_time - start_time:.5f} with {args_str}')
        return result
    return perf_clock

- decorator 미사용

In [24]:
def sum_func(x: list):
    time.sleep(1)
    return sum(x)

In [25]:
sum_func = perf_time(sum_func)

sum_func([1,2,3,4,5])

time: 1.00106 with [1, 2, 3, 4, 5]


15

- decorator 사용

In [22]:
@perf_time
def sum_func(x: list):
    time.sleep(1)
    return sum(x)

In [23]:
sum_func([1,2,3,4,5])

time: 1.00104 with [1, 2, 3, 4, 5]


15

- decorator를 사용하면 위의 미사용 부분이 저절로 진행되는 것! 
- 즉, decorator는 syntax sugar!
- 원래는 함수를 위해 고안되었지만 어떤 종류의 객체에도 적용이 가능하다.
    - 함수, 메서드, 제너레이터, 클래스 등

- decorator가 인자를 받도록 구현할 수도 있다.
    - 방법1: 중첩함수를 한 단계 더 깊게 구현
    - 방법2: decorator를 위한 class 구현 -> 이게 가독성이 더 좋다


```python
func = retry(arg1, arg2, ...)(func)
```

In [2]:
# 방법1

def with_retry(num_retry: int):
    def retry(func):
        def wrapped(*args, **kwargs):
            for _ in range(num_retry):
                try:
                    return func(*args, **kwargs)
                except:
                    print('Fail')
        return wrapped
    return retry
    
@with_retry(num_retry=5)
def tmp_func():
    pass

In [3]:
# 방법2

class WithRetry:

    def __init__(self, num_retry: int):
        self.num_retry = num_retry

    def __call__(self, func):
        def wrapped(*args, **kwargs):
            for _ in range(self.num_retry):
                try:
                    return func(*args, **kwargs)
                except:
                    print('Fail')
        return wrapped
    
@WithRetry(num_retry=5)  # 먼저, 객체가 생성되고 @연산이 호출된다.
def tmp_func():
    pass

## decorator 활용시 유의할 점

- `@wraps(func)`

In [16]:
# decorator

def trace(func):
    def wrapped(*args, **kwargs):
        print("tracing...")
        return func(*args, **kwargs)
    return wrapped

In [17]:
@trace
def func1(id):
    """func1의 docstring"""
    print(id)

In [18]:
help(func1)

Help on function wrapped in module __main__:

wrapped(*args, **kwargs)



In [19]:
func1?

Signature: func1(*args, **kwargs)
Docstring: <no docstring>
File:      /tmp/ipykernel_9773/1200381661.py
Type:      function

In [6]:
func1.__qualname__

'trace.<locals>.wrapped'

- 위처럼 우리가 의도하는 방향과 다른 결과가 나온다
- 이를 해결하기 위해서는 `@wraps`를 이용한다.

In [20]:
# decorator

from functools import wraps

def trace(func):
    @wraps(func)
    def wrapped(*args, **kwargs):
        print("tracing...")
        return func(*args, **kwargs)
    return wrapped

In [21]:
@trace
def func1(id):
    """func1의 docstring"""
    print(id)

In [22]:
help(func1)

Help on function func1 in module __main__:

func1(id)
    func1의 docstring



In [23]:
func1?

Signature: func1(id)
Docstring: func1의 docstring
File:      /tmp/ipykernel_9773/736433078.py
Type:      function

In [24]:
func1.__qualname__

'func1'

## decorator 관련 팁

- decorator가 적어도 3회 이상 필요한 경우에 구현한다. 
- 처음부터 무작정 만들지 말고 패턴이 생기고 decorator 구현에 대한 추상화가 명확해지면 만들자.
- 하나의 decorator는 하나의 기능을 갖도록 구현한다.
- 함수를 위한 decorator를 만드는 경우 가능한 경우 원래의 함수 signature와 일치하도록 만든다. `*args`, `**kwargs` 보다 원본 함수와 유사하게 만들면 유지보수가 쉽다.

- 캡슐화와 관심사의 분리
    - 실제로 하는 일과 데코레이팅하는 일의 책임을 명확히 구분해야 함
    - 데코레이터 사용자는 데코레이터의 내부구조를 알 필요 없이 블랙박스로 동작해야 함
- 독립성
    - 데코레이터가 하는 일은 독립적이여야 함
    - 테코레이팅되는 객체와 최대한 분리되어야 함
- 재사용성
    - 하나의 함수 인스턴스가 아니라 범용적이어야 함